# Ground Truth and Step Count Merge

## Setup

In [54]:
import pandas as pd
import numpy as np

In [55]:
import warnings
warnings.filterwarnings("ignore")

## Load and Check Data Integrity

**Coding Ground Truth**

In [56]:
# Import coding ground truth
ground_truth = pd.read_csv("ACT24_groundtruth_secbysec.csv", dtype={"work_type": "string"}) #dtype since missing values were stored as NaN, should be as pd.na

In [57]:
# Display first 5 rows for inspection
ground_truth.head()

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity
0,102,1,2019-07-24,2019-07-24 08:20:19,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
1,102,1,2019-07-24,2019-07-24 08:20:20,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
2,102,1,2019-07-24,2019-07-24 08:20:21,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
3,102,1,2019-07-24,2019-07-24 08:20:22,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
4,102,1,2019-07-24,2019-07-24 08:20:23,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light


In [58]:
# Check for duplicate timestamps
check = ground_truth[["id", "observation", "date_time"]].value_counts()
check[check > 1]

Series([], Name: count, dtype: int64)

**Steps Ground Truth**

In [59]:
# Import seconds ground truth
seconds = pd.read_csv("seconds_ground_truth.csv")

In [60]:
# Display first 5 rows for inspection
seconds.sort_values(["ID", "Session", "Date", "Time"]).head()

,ID,Session,Relative Time,Date,Time,Quality,Step
114078,102,1,0:00:01,7/24/2019,08:20:20,Codable,0
114079,102,1,0:00:02,7/24/2019,08:20:21,Codable,0
114080,102,1,0:00:03,7/24/2019,08:20:22,Codable,0
114081,102,1,0:00:04,7/24/2019,08:20:23,Codable,0
114082,102,1,0:00:05,7/24/2019,08:20:24,Codable,1


In [61]:
# Check for duplicate timestamps
check = seconds[["ID", "Session", "Date", "Time"]].value_counts()
# Duplicate timestamps were found, check for specific rows and step counts
check[check > 1].reset_index().merge(right = seconds, on=["ID", "Session", "Date", "Time"])

,ID,Session,Date,Time,count,Relative Time,Quality,Step
0,117,1,8/20/2019,05:36:54,2,2:19:08,Codable,0
1,117,1,8/20/2019,05:36:54,2,2:19:08,Codable,0
2,127,1,9/5/2019,04:24:03,2,0:16:07,Codable,0
3,127,1,9/5/2019,04:24:03,2,0:16:07,Codable,0
4,127,1,9/5/2019,04:24:10,2,0:16:14,Codable,0
...,...,...,...,...,...,...,...,...
152065,133,1,10/10/2019,04:02:10,2,1:30:14,Codable,1
152066,133,1,10/10/2019,03:37:28,2,1:05:32,Codable,0
152067,133,1,10/10/2019,03:37:28,2,1:05:32,Codable,0
152068,127,1,9/5/2019,04:30:03,2,0:22:07,Codable,0


## Ground Truth Data Cleaning

In [62]:
def quality_agg(srs):
    if len(srs) > 1:
        return "Ambiguous"
    return srs.iloc[0]

In [63]:
seconds_2 = seconds.copy()
# Rename columns
seconds_2.rename(columns={"ID": "id", "Session": "observation"}, inplace=True)
# Remove duplicates
# Logic: Get max step count for each timestamp
seconds_2 = seconds_2.groupby(["id", "observation", "Date", "Time", "Relative Time"]).agg({'Quality': quality_agg, 'Step': 'max'}).reset_index()
# Create datetime column
seconds_2["date_time"] = pd.to_datetime(seconds_2['Date'] + ' ' + seconds_2['Time'])
seconds_2

,id,observation,Date,Time,Relative Time,Quality,Step,date_time
0,102,1,7/24/2019,08:20:20,0:00:01,Codable,0,2019-07-24 08:20:20
1,102,1,7/24/2019,08:20:21,0:00:02,Codable,0,2019-07-24 08:20:21
2,102,1,7/24/2019,08:20:22,0:00:03,Codable,0,2019-07-24 08:20:22
3,102,1,7/24/2019,08:20:23,0:00:04,Codable,0,2019-07-24 08:20:23
4,102,1,7/24/2019,08:20:24,0:00:05,Codable,1,2019-07-24 08:20:24
...,...,...,...,...,...,...,...,...
376600,154,2,2/23/2020,01:06:29,0:02:20,Codable,1,2020-02-23 01:06:29
376601,154,2,2/23/2020,01:06:30,0:02:21,Codable,2,2020-02-23 01:06:30
376602,154,2,2/23/2020,01:06:31,0:02:22,Codable,2,2020-02-23 01:06:31
376603,154,2,2/23/2020,01:06:32,0:02:23,Codable,1,2020-02-23 01:06:32


In [64]:
# Check for duplicates again
check = seconds_2[["id", "observation", "date_time"]].value_counts()
check[check > 1]

Series([], Name: count, dtype: int64)

In [65]:
ground_truth_2 = ground_truth.copy()
# Create datetime column
ground_truth_2["date_time"] = pd.to_datetime(ground_truth["date_time"])
ground_truth_2

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity
0,102,1,2019-07-24,2019-07-24 08:20:19,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
1,102,1,2019-07-24,2019-07-24 08:20:20,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
2,102,1,2019-07-24,2019-07-24 08:20:21,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
3,102,1,2019-07-24,2019-07-24 08:20:22,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
4,102,1,2019-07-24,2019-07-24 08:20:23,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
...,...,...,...,...,...,...,...,...,...,...,...
511814,154,2,2020-02-23,2020-02-23 15:55:56,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary
511815,154,2,2020-02-23,2020-02-23 15:55:57,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary
511816,154,2,2020-02-23,2020-02-23 15:55:58,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary
511817,154,2,2020-02-23,2020-02-23 15:55:59,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary


## Check Start and End Times of Both Files

In [66]:
# Function to get start and end info for each id and session
def start_end(GT, id, session): 
    subset = GT[(GT["id"] == id) & (GT["observation"] == session)].reset_index(drop=True)
    start = min(subset["date_time"])
    end = max(subset["date_time"])
    # for i, row in subset.iterrows():
    #     if i != 0:
    #         if row["date_time"] < start:
    #             start = row["date_time"]
    #         elif row["date_time"] > end:
    #             end = row["date_time"]
    return [start, end]
    #print(f"ID = {id}, Session = {session}, Start = {start}, End = {end}")
        

In [67]:
# Function to apply start_end to entire dataframe (ie. each id and session)
def start_time_dataframe(df):
    id_ses = df.groupby(["id", "observation"]).size().reset_index().rename(columns={0: "count"})
    times = {"ID": [], 
                   "Session": [],
                   "Start": [],
                   "End": []}
    for i, row in id_ses.iterrows():
        times["ID"].append(row["id"])
        times["Session"].append(row["observation"])
        ls = start_end(df, row["id"], row["observation"])
        times["Start"].append(ls[0])
        times["End"].append(ls[1])
    return pd.DataFrame(times)

In [68]:
# Get start and end times from both coding and step ground truths
GT_times = start_time_dataframe(ground_truth_2)
seconds_times = start_time_dataframe(seconds_2)

In [69]:
start_end_info = pd.merge(GT_times, seconds_times, on=["ID", "Session"], how="outer", suffixes=("_GT", '_secondsFile'))
# Get differences between seconds/coding ground truth, in seconds
start_end_info["StartDiffSecs(GT-seconds)"] = (start_end_info["Start_GT"] - start_end_info["Start_secondsFile"]).dt.total_seconds()
start_end_info["EndDiffSecs(GT-seconds)"] = (start_end_info["End_GT"] - start_end_info["End_secondsFile"]).dt.total_seconds()
start_end_info

,ID,Session,Start_GT,End_GT,Start_secondsFile,End_secondsFile,StartDiffSecs(GT-seconds),EndDiffSecs(GT-seconds)
0,102,1,2019-07-24 08:20:19,2019-07-24 11:21:13,2019-07-24 08:20:20,2019-07-24 11:05:37,-1.0,936.0
1,102,2,2019-07-25 12:42:08,2019-07-25 15:42:00,2019-07-25 12:42:09,2019-07-25 15:28:35,-1.0,805.0
2,116,1,2019-08-20 08:08:45,2019-08-20 11:09:00,2019-08-20 08:08:46,2019-08-20 11:08:56,-1.0,4.0
3,116,2,2019-08-21 11:10:12,2019-08-21 13:28:00,2019-08-21 11:09:48,2019-08-21 13:26:58,24.0,62.0
4,117,1,2019-08-20 15:17:46,2019-08-20 18:18:00,2019-08-20 03:17:47,2019-08-20 06:05:55,43199.0,43925.0
5,117,2,2019-08-21 07:11:45,2019-08-21 10:13:00,2019-08-21 07:11:46,2019-08-21 09:41:43,-1.0,1877.0
6,122,1,2019-08-28 10:06:41,2019-08-28 13:07:00,2019-08-28 10:06:42,2019-08-28 12:42:21,-1.0,1479.0
7,122,2,2019-08-29 15:19:51,2019-08-29 18:23:00,2019-08-29 03:19:52,2019-08-29 06:22:41,43199.0,43219.0
8,124,1,2019-08-28 14:59:09,2019-08-28 17:59:05,2019-08-28 02:59:10,2019-08-28 04:49:57,43199.0,47348.0
9,124,2,2019-08-29 09:31:00,2019-08-29 12:31:00,2019-08-29 09:31:01,2019-08-29 12:19:15,-1.0,705.0


**Add Issue Flags in Columns**

In [70]:
# Function that flags id/observation pairs if the times between step and coding ground truth are off by around 12 hrs
def flag_military(row):
    if row > 40000:
        return True
    else:
        return False

In [71]:
start_end_info_2 = start_end_info.copy()
start_end_info_2["MilitaryTimeFlag"] = start_end_info_2["StartDiffSecs(GT-seconds)"].apply(flag_military)
start_end_info_2

,ID,Session,Start_GT,End_GT,Start_secondsFile,End_secondsFile,StartDiffSecs(GT-seconds),EndDiffSecs(GT-seconds),MilitaryTimeFlag
0,102,1,2019-07-24 08:20:19,2019-07-24 11:21:13,2019-07-24 08:20:20,2019-07-24 11:05:37,-1.0,936.0,False
1,102,2,2019-07-25 12:42:08,2019-07-25 15:42:00,2019-07-25 12:42:09,2019-07-25 15:28:35,-1.0,805.0,False
2,116,1,2019-08-20 08:08:45,2019-08-20 11:09:00,2019-08-20 08:08:46,2019-08-20 11:08:56,-1.0,4.0,False
3,116,2,2019-08-21 11:10:12,2019-08-21 13:28:00,2019-08-21 11:09:48,2019-08-21 13:26:58,24.0,62.0,False
4,117,1,2019-08-20 15:17:46,2019-08-20 18:18:00,2019-08-20 03:17:47,2019-08-20 06:05:55,43199.0,43925.0,True
5,117,2,2019-08-21 07:11:45,2019-08-21 10:13:00,2019-08-21 07:11:46,2019-08-21 09:41:43,-1.0,1877.0,False
6,122,1,2019-08-28 10:06:41,2019-08-28 13:07:00,2019-08-28 10:06:42,2019-08-28 12:42:21,-1.0,1479.0,False
7,122,2,2019-08-29 15:19:51,2019-08-29 18:23:00,2019-08-29 03:19:52,2019-08-29 06:22:41,43199.0,43219.0,True
8,124,1,2019-08-28 14:59:09,2019-08-28 17:59:05,2019-08-28 02:59:10,2019-08-28 04:49:57,43199.0,47348.0,True
9,124,2,2019-08-29 09:31:00,2019-08-29 12:31:00,2019-08-29 09:31:01,2019-08-29 12:19:15,-1.0,705.0,False


**Issues:** 
* Negative start second differences implies that the ground truth start times are OUTSIDE the seconds file session times
* Positive end second differences implies that the ground truth end times are OUTSIDE the seconds file session times

**This implies that we have tagged times for this (ID, Session) pair in our ground truth file that have no information about steps taken :(**

**1\. Ground Truth File contains military time timestamps but seconds file is in AM/PM**
* MilitaryTimeFlag = True means this row has this problem
* **NOTE**: Fixed.

**2\. (ID, Session) pairs missing in seconds file (9 total)**
* Probably just need to throw these out since we do not have step info for these :(

## Fix AM/PM Disparity

In [72]:
start_end_info_3 = start_end_info_2.copy()
# Fix start AM/PM
start_end_info_3['Start_secondsFile'] = start_end_info_3['Start_secondsFile'] + pd.to_timedelta("12:00:00") * start_end_info_3['MilitaryTimeFlag']
# Fix end AM/PM
start_end_info_3['End_secondsFile'] = start_end_info_3['End_secondsFile'] + pd.to_timedelta("12:00:00") * start_end_info_3['MilitaryTimeFlag']
# Recalculate differences
start_end_info_3["StartDiffSecs(GT-seconds)"] = (start_end_info_3["Start_GT"] - start_end_info_3["Start_secondsFile"]).dt.total_seconds()
start_end_info_3["EndDiffSecs(GT-seconds)"] = (start_end_info_3["End_GT"] - start_end_info_3["End_secondsFile"]).dt.total_seconds()
# Recode military time flag
start_end_info_3['MilitaryTimeFlag'] = start_end_info_3["StartDiffSecs(GT-seconds)"].apply(flag_military)
# We need to determine which rows in the second-by-second data fall within the intersection of the groundtruth time intervals, since they don't match up
# Find the greatest start time
start_end_info_3['max_start'] = start_end_info_3[['Start_GT', 'Start_secondsFile']].max(axis=1)
# Find the smallest end time
start_end_info_3['min_end'] = start_end_info_3[['End_GT', 'End_secondsFile']].min(axis=1)
start_end_info_3

,ID,Session,Start_GT,End_GT,Start_secondsFile,End_secondsFile,StartDiffSecs(GT-seconds),EndDiffSecs(GT-seconds),MilitaryTimeFlag,max_start,min_end
0,102,1,2019-07-24 08:20:19,2019-07-24 11:21:13,2019-07-24 08:20:20,2019-07-24 11:05:37,-1.0,936.0,False,2019-07-24 08:20:20,2019-07-24 11:05:37
1,102,2,2019-07-25 12:42:08,2019-07-25 15:42:00,2019-07-25 12:42:09,2019-07-25 15:28:35,-1.0,805.0,False,2019-07-25 12:42:09,2019-07-25 15:28:35
2,116,1,2019-08-20 08:08:45,2019-08-20 11:09:00,2019-08-20 08:08:46,2019-08-20 11:08:56,-1.0,4.0,False,2019-08-20 08:08:46,2019-08-20 11:08:56
3,116,2,2019-08-21 11:10:12,2019-08-21 13:28:00,2019-08-21 11:09:48,2019-08-21 13:26:58,24.0,62.0,False,2019-08-21 11:10:12,2019-08-21 13:26:58
4,117,1,2019-08-20 15:17:46,2019-08-20 18:18:00,2019-08-20 15:17:47,2019-08-20 18:05:55,-1.0,725.0,False,2019-08-20 15:17:47,2019-08-20 18:05:55
5,117,2,2019-08-21 07:11:45,2019-08-21 10:13:00,2019-08-21 07:11:46,2019-08-21 09:41:43,-1.0,1877.0,False,2019-08-21 07:11:46,2019-08-21 09:41:43
6,122,1,2019-08-28 10:06:41,2019-08-28 13:07:00,2019-08-28 10:06:42,2019-08-28 12:42:21,-1.0,1479.0,False,2019-08-28 10:06:42,2019-08-28 12:42:21
7,122,2,2019-08-29 15:19:51,2019-08-29 18:23:00,2019-08-29 15:19:52,2019-08-29 18:22:41,-1.0,19.0,False,2019-08-29 15:19:52,2019-08-29 18:22:41
8,124,1,2019-08-28 14:59:09,2019-08-28 17:59:05,2019-08-28 14:59:10,2019-08-28 16:49:57,-1.0,4148.0,False,2019-08-28 14:59:10,2019-08-28 16:49:57
9,124,2,2019-08-29 09:31:00,2019-08-29 12:31:00,2019-08-29 09:31:01,2019-08-29 12:19:15,-1.0,705.0,False,2019-08-29 09:31:01,2019-08-29 12:19:15


In [73]:
# Check to make sure there are no more AM/PM disparities
start_end_info_3[start_end_info_3['MilitaryTimeFlag']]

,ID,Session,Start_GT,End_GT,Start_secondsFile,End_secondsFile,StartDiffSecs(GT-seconds),EndDiffSecs(GT-seconds),MilitaryTimeFlag,max_start,min_end


In [74]:
# Edit steps ground truth to fix AM/PM disparities
seconds_3 = seconds_2.merge(right = start_end_info_2[["ID", "Session", "MilitaryTimeFlag", "StartDiffSecs(GT-seconds)"]], how = 'left',
              left_on = ["id", "observation"], right_on = ["ID", "Session"])
seconds_3["date_time"] = seconds_3["date_time"] + pd.to_timedelta("12:00:00") * seconds_3["MilitaryTimeFlag"]
seconds_3 = seconds_3[["id", "observation", "date_time", "Quality", "Step"]]
seconds_3

,id,observation,date_time,Quality,Step
0,102,1,2019-07-24 08:20:20,Codable,0
1,102,1,2019-07-24 08:20:21,Codable,0
2,102,1,2019-07-24 08:20:22,Codable,0
3,102,1,2019-07-24 08:20:23,Codable,0
4,102,1,2019-07-24 08:20:24,Codable,1
...,...,...,...,...,...
376600,154,2,2020-02-23 13:06:29,Codable,1
376601,154,2,2020-02-23 13:06:30,Codable,2
376602,154,2,2020-02-23 13:06:31,Codable,2
376603,154,2,2020-02-23 13:06:32,Codable,1


In [75]:
# Check for duplicate times
check = seconds_3[["id", "observation", "date_time"]].value_counts()
check[check>1]

Series([], Name: count, dtype: int64)

## Merge Ground Truth Datasets

In [76]:
# Outer merge coding with steps ground truth
merged = pd.merge(left = ground_truth_2, right = seconds_3.drop("Quality", axis=1), how="outer", on=["id", "observation", "date_time"])
# Add flag for times that are inside the max start and min end interval
merged = merged.merge(right = start_end_info_3[["ID", "Session", "max_start", "min_end"]], how = 'left',
                            left_on = ["id", "observation"], right_on = ["ID", "Session"])
merged["inside_flag"] = (merged["date_time"] >= merged["max_start"]) & (merged["date_time"] <= merged["min_end"])
merged = merged.drop(["ID", "Session", "max_start", "min_end"], axis=1)
merged

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,Step,inside_flag
0,102,1,2019-07-24,2019-07-24 08:20:19,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,NaN,False
1,102,1,2019-07-24,2019-07-24 08:20:20,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
2,102,1,2019-07-24,2019-07-24 08:20:21,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
3,102,1,2019-07-24,2019-07-24 08:20:22,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
4,102,1,2019-07-24,2019-07-24 08:20:23,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511919,154,2,2020-02-23,2020-02-23 15:55:56,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False
511920,154,2,2020-02-23,2020-02-23 15:55:57,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False
511921,154,2,2020-02-23,2020-02-23 15:55:58,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False
511922,154,2,2020-02-23,2020-02-23 15:55:59,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False


In [77]:
# Check to make sure that there is only one row per second
check = merged[["id", "observation", "date_time"]].value_counts()
check[check > 1]

Series([], Name: count, dtype: int64)

## Check Valid Rows (Intersection of Steps, Coding Tables)

At this point, we know that the rows are invalid if:

1. The ID and Session/observation are missing from the seconds ground truth entirely, not partially
2. OR They fall outside of where the time intervals of the two ground truth files intersect.

We need to know if any more rows are invalid. The only rows that could be invalid at this point are rows that:

1. Have a valid ID/Session
2. AND they fall within the time interval
3. AND they either don't have steps or coding ground truth

We need to check if any such rows exist.

In [78]:
# List of id/session that were found in the seconds file and in ground truth file
id_session_notna = start_end_info_3[["ID", "Session"]][~start_end_info_3["Start_secondsFile"].isna()]
id_session_notna

,ID,Session
0,102,1
1,102,2
2,116,1
3,116,2
4,117,1
5,117,2
6,122,1
7,122,2
8,124,1
9,124,2


In [79]:
# Find all valid rows - rows that should have data from both coding and step ground truth
# They should be in the valid id/sessions
merged_valid = id_session_notna.merge(right = merged, left_on = ["ID", "Session"], right_on = ["id", "observation"]).drop(["ID", "Session"], axis=1)
# They should also fall within the time interval between the max start and min end of both coding/step ground truth
merged_valid = merged_valid[merged_valid["inside_flag"] == True]
merged_valid

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,Step,inside_flag
1,102,1,2019-07-24,2019-07-24 08:20:20,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
2,102,1,2019-07-24,2019-07-24 08:20:21,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
3,102,1,2019-07-24,2019-07-24 08:20:22,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
4,102,1,2019-07-24,2019-07-24 08:20:23,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
5,102,1,2019-07-24,2019-07-24 08:20:24,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
404517,154,2,2020-02-23,2020-02-23 13:06:29,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,1.0,True
404518,154,2,2020-02-23,2020-02-23 13:06:30,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,2.0,True
404519,154,2,2020-02-23,2020-02-23 13:06:31,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,2.0,True
404520,154,2,2020-02-23,2020-02-23 13:06:32,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,1.0,True


In [80]:
# Check if there are any invalid rows remaining - there should be none
merged_valid[(merged_valid["walking_not"].isna()) | (merged_valid["Step"].isna())]

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,Step,inside_flag


Based on our analysis, there are no invalid rows that:

1. Have a valid ID/Session
2. AND they fall within the time interval
3. AND they either don't have steps or coding ground truth

## Analyze Time Interval Disparity

Here we show the rows that have a valid ID/Session but fall outside the time interval intersection of the two ground truth tables. They should match the sum of the difference in seconds of the start/end that we calculated in the start_end_info tables.

In [81]:
# Find the total number of seconds/rows that fall outside the max start/min end interval
total_outside = start_end_info_3['StartDiffSecs(GT-seconds)'].abs().sum() + start_end_info_3['EndDiffSecs(GT-seconds)'].abs().sum()
total_outside

38189.0

In [82]:
merged_outside = merged[merged["inside_flag"] == False]
merged_outside

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,Step,inside_flag
0,102,1,2019-07-24,2019-07-24 08:20:19,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,NaN,False
9919,102,1,2019-07-24,2019-07-24 11:05:38,WRK- general,work_education,"SP- Office (business, professional services, f...",walk,not_sedentary,walking,moderate,NaN,False
9920,102,1,2019-07-24,2019-07-24 11:05:39,WRK- screen based,work_education,"SP- Office (business, professional services, f...",sitting,sedentary,not_walking,sedentary,NaN,False
9921,102,1,2019-07-24,2019-07-24 11:05:40,WRK- screen based,work_education,"SP- Office (business, professional services, f...",sitting,sedentary,not_walking,sedentary,NaN,False
9922,102,1,2019-07-24,2019-07-24 11:05:41,WRK- screen based,work_education,"SP- Office (business, professional services, f...",sitting,sedentary,not_walking,sedentary,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511919,154,2,2020-02-23,2020-02-23 15:55:56,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False
511920,154,2,2020-02-23,2020-02-23 15:55:57,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False
511921,154,2,2020-02-23,2020-02-23 15:55:58,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False
511922,154,2,2020-02-23,2020-02-23 15:55:59,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False


The numbers match up. We have the correct number of rows that fall outside the interval. Because they fall outside of the interval intersection, they will only have steps or coding ground truth, not both.

We can further separate the two into rows that are in only coding and rows that are only in step count (that fall outside the common time interval intersection).

In [83]:
# Only in step count
sc_only = merged_outside[merged_outside["activity_type"].isna()]
sc_only

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,Step,inside_flag
32464,116,2,NaN,2019-08-21 11:09:48,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.0,False
32465,116,2,NaN,2019-08-21 11:09:49,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.0,False
32466,116,2,NaN,2019-08-21 11:09:50,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.0,False
32467,116,2,NaN,2019-08-21 11:09:51,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.0,False
32468,116,2,NaN,2019-08-21 11:09:52,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
480064,150,2,NaN,2020-01-21 19:31:54,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.0,False
480065,150,2,NaN,2020-01-21 19:31:55,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.0,False
480066,150,2,NaN,2020-01-21 19:31:56,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.0,False
480067,150,2,NaN,2020-01-21 19:31:57,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.0,False


In [84]:
# Only in coding
gt_only = merged_outside[merged_outside["Step"].isna()]
gt_only

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,Step,inside_flag
0,102,1,2019-07-24,2019-07-24 08:20:19,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,NaN,False
9919,102,1,2019-07-24,2019-07-24 11:05:38,WRK- general,work_education,"SP- Office (business, professional services, f...",walk,not_sedentary,walking,moderate,NaN,False
9920,102,1,2019-07-24,2019-07-24 11:05:39,WRK- screen based,work_education,"SP- Office (business, professional services, f...",sitting,sedentary,not_walking,sedentary,NaN,False
9921,102,1,2019-07-24,2019-07-24 11:05:40,WRK- screen based,work_education,"SP- Office (business, professional services, f...",sitting,sedentary,not_walking,sedentary,NaN,False
9922,102,1,2019-07-24,2019-07-24 11:05:41,WRK- screen based,work_education,"SP- Office (business, professional services, f...",sitting,sedentary,not_walking,sedentary,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511919,154,2,2020-02-23,2020-02-23 15:55:56,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False
511920,154,2,2020-02-23,2020-02-23 15:55:57,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False
511921,154,2,2020-02-23,2020-02-23 15:55:58,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False
511922,154,2,2020-02-23,2020-02-23 15:55:59,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,NaN,False


Some further analysis on the rows that are only in the coding ground truth:

In [85]:
gt_only_summary = gt_only['activity_type'].value_counts()
gt_only_summary = gt_only_summary.reset_index()
gt_only_summary = gt_only_summary.rename(columns = {"activity_type": "category"})
gt_only_summary['column'] = 'activity_type'

for col in ['broad_activity_type', 'work_type', 'posture', 'sedentary_not', 'walking_not']:
    new_summary = gt_only[col].value_counts()
    new_summary = new_summary.reset_index()
    new_summary = new_summary.rename(columns = {col: "category"})
    new_summary['column'] = col
    gt_only_summary = pd.concat([gt_only_summary, new_summary], axis=0)
    
gt_only_summary.groupby(["column", "category"]).sum()

count
column              category                                                 
activity_type       CA- caring for and helping children                   384
                    EAT- eating and drinking, waiting                    9488
                    EDU- taking class, research, homework               11759
                    EX- other                                               1
                    EX- surfing/water sport                                 1
                    EX- weight training                                     1
                    HA- food prep and cleanup                               1
                    HA- household management/other household activi...    436
                    HA- lawn, garden and houseplants                       19
                    LES- screen based leisure time (TV, video game,...   3041
                    LES- socializing, communicating, leisure time n...   3212
                    OTHER- non codable                                    167
                    SL- sleep                                               1
                    TRAV- passenger (car/truck/motorcycle)                  1
                    TRAV- walking                                           4
                    WRK- general                                          929
                    WRK- screen based                                    8639
broad_activity_type active_transportation                                   4
                    exercise                                                3
                    household_personal                                    822
                    lawn_garden                                            19
                    leisure                                             15741
                    non_codable                                           167
                    transportation                                          1
                    work_education                                      21327
posture             kneeling/ squatting                                    56
                    lying                                                   1
                    muscle strengthening                                    1
                    private/not coded                                     234
                    sitting                                             37023
                    stand                                                 512
                    stand and move                                        195
                    walk                                                   59
                    walk with load                                          3
sedentary_not       non_codable                                           234
                    not_sedentary                                         770
                    sedentary                                           37080
walking_not         non_codable                                           234
                    not_walking                                         37788
                    walking                                                62
work_type           Other                                                   1
                    SP- Education and Health Services                    2644
                    SP- Leisure and Hospiltality                          638
                    SP- Office (business, professional services, fi...   6122
                    SP- Trade, Retail, Transportation, and Utilities      163

**IMPORTANT NOTE**: Observed, most rows are non-walking, sedentary, sitting. We may re-add these rows to the dataset if we can confirm from Dr. Keadle that we can assume a step count of 0 for these rows.

Some further analysis on the rows only in step count:

In [86]:
sc_only['Step'].value_counts()

Step
0.0    81
1.0    15
2.0     8
3.0     1
Name: count, dtype: int64

## UPDATE: Impute Behavioral-Only with 0s and Re-Add to Data

All behavioral only rows have been determined to be 0 steps. We will impute and add back to the ground truth.

In [87]:
# Impute with 0s
gt_only2 = gt_only.copy()
gt_only2["Step"] = 0

# Add back to dataset
merged_valid2 = pd.concat([merged_valid, gt_only2], ignore_index=True).sort_values(["id", "observation", "date_time"])
merged_valid2

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,Step,inside_flag
376500,102,1,2019-07-24,2019-07-24 08:20:19,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,False
0,102,1,2019-07-24,2019-07-24 08:20:20,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
1,102,1,2019-07-24,2019-07-24 08:20:21,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
2,102,1,2019-07-24,2019-07-24 08:20:22,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
3,102,1,2019-07-24,2019-07-24 08:20:23,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
414579,154,2,2020-02-23,2020-02-23 15:55:56,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,0.0,False
414580,154,2,2020-02-23,2020-02-23 15:55:57,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,0.0,False
414581,154,2,2020-02-23,2020-02-23 15:55:58,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,0.0,False
414582,154,2,2020-02-23,2020-02-23 15:55:59,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,0.0,False


## Analyze Missing IDs/Sessions from Steps Ground Truth

These rows correspond to coding ground truth ID/Sessions that are missing from the steps ground truth entirely.

In [88]:
# List of id/session that were found in ground truth file but NOT seconds file
id_session_na = start_end_info_3[["ID", "Session"]][start_end_info_3["Start_secondsFile"].isna()]
id_session_na

,ID,Session
10,126,1
14,128,1
27,134,2
31,136,2
33,138,2
34,139,1
35,139,2
40,143,1
46,154,1


In [89]:
merged_invalid_idsession = id_session_na.merge(right = merged, left_on = ["ID", "Session"], right_on = ["id", "observation"]).drop(["ID", "Session"], axis=1)
merged_invalid_idsession

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,Step,inside_flag
0,126,1,2019-09-02,2019-09-02 11:14:51,"LES- socializing, communicating, leisure time ...",leisure,<NA>,sitting,sedentary,not_walking,sedentary,NaN,True
1,126,1,2019-09-02,2019-09-02 11:14:52,"LES- socializing, communicating, leisure time ...",leisure,<NA>,sitting,sedentary,not_walking,sedentary,NaN,True
2,126,1,2019-09-02,2019-09-02 11:14:53,"LES- socializing, communicating, leisure time ...",leisure,<NA>,sitting,sedentary,not_walking,sedentary,NaN,True
3,126,1,2019-09-02,2019-09-02 11:14:54,"LES- socializing, communicating, leisure time ...",leisure,<NA>,sitting,sedentary,not_walking,sedentary,NaN,True
4,126,1,2019-09-02,2019-09-02 11:14:55,"LES- socializing, communicating, leisure time ...",leisure,<NA>,sitting,sedentary,not_walking,sedentary,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97230,154,1,2020-02-22,2020-02-22 16:06:55,TRAV- walking,active_transportation,<NA>,walk,not_sedentary,walking,moderate,NaN,True
97231,154,1,2020-02-22,2020-02-22 16:06:56,TRAV- walking,active_transportation,<NA>,walk,not_sedentary,walking,moderate,NaN,True
97232,154,1,2020-02-22,2020-02-22 16:06:57,TRAV- walking,active_transportation,<NA>,walk,not_sedentary,walking,moderate,NaN,True
97233,154,1,2020-02-22,2020-02-22 16:06:58,TRAV- walking,active_transportation,<NA>,walk,not_sedentary,walking,moderate,NaN,True


## New Time Intervals

In [90]:
final_start_end = start_time_dataframe(merged_valid2)
final_start_end

,ID,Session,Start,End
0,102,1,2019-07-24 08:20:19,2019-07-24 11:21:13
1,102,2,2019-07-25 12:42:08,2019-07-25 15:42:00
2,116,1,2019-08-20 08:08:45,2019-08-20 11:09:00
3,116,2,2019-08-21 11:10:12,2019-08-21 13:28:00
4,117,1,2019-08-20 15:17:46,2019-08-20 18:18:00
5,117,2,2019-08-21 07:11:45,2019-08-21 10:13:00
6,122,1,2019-08-28 10:06:41,2019-08-28 13:07:00
7,122,2,2019-08-29 15:19:51,2019-08-29 18:23:00
8,124,1,2019-08-28 14:59:09,2019-08-28 17:59:05
9,124,2,2019-08-29 09:31:00,2019-08-29 12:31:00


Compare to log:

In [91]:
log = pd.read_csv("do_log_final.csv")
log["Start"] = pd.to_datetime(log["start_month"].astype(str) + "/" +
                              log["start_day"].astype(str) + "/" +
                              log["start_year"].astype(str) + "/" + ' ' +
                              log["start_time"])
log["End"] = pd.to_datetime(log["start_month"].astype(str) + "/" +
                              log["start_day"].astype(str) + "/" +
                              log["start_year"].astype(str) + "/" + ' ' +
                              log["stop_time"])
log = log[['id', 'do', 'Start', 'End']].rename(columns = {"do": "Session", "id": "ID"})
log

,ID,Session,Start,End
0,102,1,2019-07-24 08:20:19,2019-07-24 11:21:13
1,102,2,2019-07-25 12:42:08,2019-07-25 15:42:00
2,116,1,2019-08-20 08:08:45,2019-08-20 11:09:00
3,116,2,2019-08-21 11:09:47,2019-08-21 13:28:00
4,117,1,2019-08-20 15:17:46,2019-08-20 18:18:00
5,117,2,2019-08-21 07:11:45,2019-08-21 10:13:00
6,122,1,2019-08-28 10:06:41,2019-08-28 13:07:00
7,122,2,2019-08-29 15:19:51,2019-08-29 18:23:00
8,124,1,2019-08-28 14:59:09,2019-08-28 17:59:05
9,124,2,2019-08-29 09:31:00,2019-08-29 12:31:00


In [92]:
log_compare = final_start_end.merge(log, on=["ID", "Session"], how="outer", suffixes=("_final", "_log"))
log_compare["Start_diff"] = (log_compare["Start_final"] - log_compare["Start_log"]).dt.total_seconds()
log_compare["End_diff"] = (log_compare["End_final"] - log_compare["End_log"]).dt.total_seconds()
log_compare

,ID,Session,Start_final,End_final,Start_log,End_log,Start_diff,End_diff
0,102,1,2019-07-24 08:20:19,2019-07-24 11:21:13,2019-07-24 08:20:19,2019-07-24 11:21:13,0.0,0.0
1,102,2,2019-07-25 12:42:08,2019-07-25 15:42:00,2019-07-25 12:42:08,2019-07-25 15:42:00,0.0,0.0
2,116,1,2019-08-20 08:08:45,2019-08-20 11:09:00,2019-08-20 08:08:45,2019-08-20 11:09:00,0.0,0.0
3,116,2,2019-08-21 11:10:12,2019-08-21 13:28:00,2019-08-21 11:09:47,2019-08-21 13:28:00,25.0,0.0
4,117,1,2019-08-20 15:17:46,2019-08-20 18:18:00,2019-08-20 15:17:46,2019-08-20 18:18:00,0.0,0.0
5,117,2,2019-08-21 07:11:45,2019-08-21 10:13:00,2019-08-21 07:11:45,2019-08-21 10:13:00,0.0,0.0
6,122,1,2019-08-28 10:06:41,2019-08-28 13:07:00,2019-08-28 10:06:41,2019-08-28 13:07:00,0.0,0.0
7,122,2,2019-08-29 15:19:51,2019-08-29 18:23:00,2019-08-29 15:19:51,2019-08-29 18:23:00,0.0,0.0
8,124,1,2019-08-28 14:59:09,2019-08-28 17:59:05,2019-08-28 14:59:09,2019-08-28 17:59:05,0.0,0.0
9,124,2,2019-08-29 09:31:00,2019-08-29 12:31:00,2019-08-29 09:31:00,2019-08-29 12:31:00,0.0,0.0


In [93]:
# log export for data pull
log_exp = log_compare.copy()
log_exp["Start"] = log_exp["Start_final"].combine_first(log_exp["Start_log"])
log_exp["End"] = log_exp["End_final"].combine_first(log_exp["End_log"])
log_exp[["ID", "Session", "Start", "End"]].to_csv("log-new.csv", index=False)

## Final Table

**Final Ground Truth with Invalid Rows Dropped**

In [94]:
final_gt = merged_valid2.drop(["inside_flag"], axis=1).rename(columns={"Step": "step"})
final_gt['step'] = final_gt['step'].astype(int)
final_gt

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,step
376500,102,1,2019-07-24,2019-07-24 08:20:19,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0
0,102,1,2019-07-24,2019-07-24 08:20:20,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0
1,102,1,2019-07-24,2019-07-24 08:20:21,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0
2,102,1,2019-07-24,2019-07-24 08:20:22,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0
3,102,1,2019-07-24,2019-07-24 08:20:23,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,0
...,...,...,...,...,...,...,...,...,...,...,...,...
414579,154,2,2020-02-23,2020-02-23 15:55:56,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,0
414580,154,2,2020-02-23,2020-02-23 15:55:57,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,0
414581,154,2,2020-02-23,2020-02-23 15:55:58,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,0
414582,154,2,2020-02-23,2020-02-23 15:55:59,"EDU- taking class, research, homework",work_education,<NA>,sitting,sedentary,not_walking,sedentary,0


In [95]:
final_gt.to_csv("merged_groundtruth_secbysec.csv", index=False)